In [1]:
import glob
import pandas as pd
import os
import numpy as np
import datetime

print(datetime.datetime.now())
print(os.getcwd())



2020-04-16 15:31:05.621049
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413


In [2]:
list_files=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/*.csv")
list_files

['/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Prospect_BL_MobileVisitor_Control.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Prospect_BL_MobileVisitor_Test.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Rewards_L12_PZip_Control.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Rewards_L12_PZip_Test.csv']

In [3]:
for file in list_files:
    df=pd.read_csv(file,dtype=str,nrows=10)
    print(os.path.basename(file),df.columns.tolist())

AudienceBL_DynamicLP_Prospect_BL_MobileVisitor_Control.csv ['index', 'ad_id']
AudienceBL_DynamicLP_Prospect_BL_MobileVisitor_Test.csv ['index', 'ad_id']
AudienceBL_DynamicLP_Rewards_L12_PZip_Control.csv ['index', 'customer_id_hashed', 'max_transaction_dt', 'total_transaction', 'total_sales', 'email_address_hash', 'zip_cd', 'zip_type']
AudienceBL_DynamicLP_Rewards_L12_PZip_Test.csv ['index', 'customer_id_hashed', 'max_transaction_dt', 'total_transaction', 'total_sales', 'email_address_hash', 'zip_cd', 'zip_type']


In [4]:
list_email=[x for x in list_files if "Rewards" in x]
list_apid=[x for x in list_files if "Prospect" in x]

print("list_email",list_email)
print("list_apid",list_apid)

list_email ['/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Rewards_L12_PZip_Control.csv', '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Rewards_L12_PZip_Test.csv']
list_apid ['/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Prospect_BL_MobileVisitor_Control.csv', '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/FB_Mobile_testing/BR_audience_mobile_dynamic_landing_page_20200413/dynamic_landing_page_audience/AudienceBL_DynamicLP_Prospect_BL_MobileVisitor_Test.csv']


# FB

In [6]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi

BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBAHsJtDc00ZBtZAW2ye9CRdKsy9ivLlzvgIMsEkZAyZA3mRpuGBF8IvZAR9obyYZCdzfw455nzMEW76A5ZBZBS6Fft2nQsUSs8sZCcGDZBZCbVeiCq0LMvekWjnnDZCuglkNihTAZAWIWTpvzaZBeZCKYywrPl0ZAsoWul36cyQZDZD"


In [24]:
def creatfbaudience_from_datatype(AudienceName,AudienceDescription,df,ACCESS_TOKEN,accountid,fb_datatype,col_in_df):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v6.0')
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    audience = CustomAudience(fbname)
    users = list(df[col_in_df])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(schema=fb_datatype, users=list0)
        #print len(list0)   
    print(fbname,datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)

In [25]:
for file in list_email:
    audience_name = os.path.basename(file).split(".")[0]
    description=os.path.basename(file).split(".")[0]+"_"+str(datetime.datetime.now().date())
    df = pd.read_csv(file,dtype=str)
    creatfbaudience_from_datatype(AudienceName=audience_name,
                    AudienceDescription=description,
                    df=df,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    fb_datatype=CustomAudience.Schema.email_hash,
                    col_in_df="email_address_hash"
                   )

23844455470800265 2020-04-16 14:09:33.699761 AudienceBL_DynamicLP_Rewards_L12_PZip_Control uploaded 135000
23844455471120265 2020-04-16 14:09:46.289718 AudienceBL_DynamicLP_Rewards_L12_PZip_Test uploaded 135000


# AdWords

In [8]:
import uuid
from googleads import adwords
import os
import datetime
auth_file_path = '/home/jian/Docs/Google_Ads_Access/googleads_BL_MMC.yaml'

def upload_and_return_count_in_chunk(service, user_list_id, email_list):
    ids_in_chunk=0
    mutate_members_operation = {
      'operand': {
          'userListId': user_list_id,
          'membersList': email_list
      },
      'operator': 'ADD'
    }
    response = service.mutateMembers([mutate_members_operation])
    if 'userLists' in response:
        for user_list in response['userLists']:
            # print('Uploaded %i items to user list with name "%s" and ID "%d"'
            #     % (len(email_list), user_list['name'], user_list['id']))

            
            ids_in_chunk+=len(email_list)
    return ids_in_chunk
            
def run_the_uploading(client, email_file_path):
  # Initialize appropriate services.
    total_ids_in_seg=0
    user_list_service = client.GetService('AdwordsUserListService', 'v201809')
    file_base_name=os.path.basename(email_file_path).split(".")[0]
    user_list = {
        'xsi_type': 'CrmBasedUserList',
        'name': file_base_name+"_directly_uploaded",
        'description': 'Aggregated control for 2020Q1 RFM: '+file_base_name,
        # CRM-based user lists can use a membershipLifeSpan of 10000 to indicate
        # unlimited; otherwise normal values apply.
        'membershipLifeSpan': 10000,
        'uploadKeyType': 'CONTACT_INFO'
     }
    # Create an operation to add the user list.
    operations = [{
        'operator': 'ADD',
        'operand': user_list
    }]
    result = user_list_service.mutate(operations)
    user_list_id = result['value'][0]['id']

    df = pd.read_csv(email_file_path, chunksize=3000)
    for i in df:
        mail_col = [k for k in i.columns if 'email' in k.lower()][0]
        new_col = ['email' if h == mail_col else h for h in i.columns.tolist()]
        i.columns = new_col
        emails = i['email']
        emails = set([i for i in emails if str(i) != 'nan'])
        members = [{'hashedEmail': email} for email in emails]
        ids_in_chunk=upload_and_return_count_in_chunk(user_list_service, user_list_id, members)
        
        total_ids_in_seg+=ids_in_chunk
    print(datetime.datetime.now(),"all %s ids uploaded in the segment "%str(total_ids_in_seg))

In [9]:
BL_client = adwords.AdWordsClient.LoadFromStorage(auth_file_path)
for local_seg_file in list_email:
    run_the_uploading(BL_client, local_seg_file)

2020-04-16 14:37:19.980738 all 135000 ids uploaded in the segment 
2020-04-16 14:37:53.399590 all 135000 ids uploaded in the segment 
